# Statistics

### BUSI 520 - Python for Business Research
### Kerry Back, JGSB, Rice University

* $t$ tests with scipy.stats
* regressions with statsmodels formula API
    * HAC std errors
  * logit
* regressions with linearmodels
  * two-stage least squares
  * seemingly unrelated regressions
  * fixed effects and clustered std errors
  * Fama-MacBeth
* VARs with statsmodels
* saving output
  * to Excel
  * to latex with pystout

In [150]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr

from scipy.stats import ttest_1samp, ttest_ind 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS, FamaMacBeth
from linearmodels.iv import IV2SLS
from linearmodels.system import SUR
from statsmodels.regression.rolling import RollingOLS
from statsmodels.tsa.api import VAR
from pystout import pystout

## $t$ tests with scipy.stats

In [151]:
# use result.statistic and result.pvalue to get t-stat and p-value

sampleA = np.random.normal(size=100)
sampleB = np.random.normal(loc=0.1, scale=2, size=150)

# test of H0: mean = 0
result1 = ttest_1samp(sampleB, popmean=0)

# test of H0: meanA = meanB assuming equal variances
result2 = ttest_ind(sampleA, sampleB)

# test 0f H0: meanA = meanB allowing unequal variances
result3 = ttest_ind(sampleA, sampleB, equal_var=False)

## Regressions with statsmodels

In [152]:
data = pd.read_csv("WAGE1_revised.csv")

In [153]:
# use result.summary() 

model1 = sm.OLS(endog=data.wage, exog=sm.add_constant(data.exper))
result1 = model1.fit()

model2 = smf.ols("wage ~ female", data=data)
result2 = model2.fit()

# multivariate
model3 = smf.ols("wage ~ female + educ", data=data)
result3 = model3.fit()

# transformations of variables
model4 = smf.ols("wage ~ female + educ + np.log(exper)", data=data)
result4 = model4.fit()

In [154]:
# interactions
model5 = smf.ols("wage ~ female + educ + female*educ + np.log(exper)", data=data)
result5 = model5.fit()

# dummy variables
model6 = smf.ols(
    "wage ~ female + educ + female*educ + np.log(exper) + C(area)", 
    data=data
)
result6 = model6.fit()

# regression without an intercept
model7 = smf.ols(
    "wage ~ female + educ + female*educ + np.log(exper) + C(area) - 1", 
    data=data
)
result7 = model7.fit()

## Regressions with linearmodels

The linearmodels package was created by Kevin Sheppard.  Many of the following examples come from the linearmodels user guide https://bashtage.github.io/linearmodels/.

### Two Stage Least Squares

In [155]:
# use result.summary 

from linearmodels.datasets import wage

data = wage.load()
data = data.dropna(subset=["educ", "wage", "sibs", "exper"])

model1 = IV2SLS.from_formula("np.log(wage) ~ exper + [educ ~ sibs]", data=data)
result1 = model1.fit(cov_type="robust")


### Seemingly Unrelated Regressions

In [156]:
from linearmodels.datasets import fringe
data = fringe.load()

formula = """ 
    {hrbens ~ educ + exper + union + south + nrtheast + nrthcen + male}
    {hrearn ~ educ + exper + nrtheast + married + male}
    """
model2 = SUR.from_formula(formula, data=data)
result2 = model2.fit(cov_type="robust")

### Panel Regressions with Fixed Effects and Clustered Standard Errors

To use the formula version of linearmodels with fixed effects, create a multi-index for the dataframe with the outside (first) index being "entity" and the inside (second) index being "time."  For other fixed effects, use the basic (non-formula) version of linearmodels.

In [157]:
from linearmodels.datasets import wage_panel
data = wage_panel.load()
data = data.set_index(["nr", "year"])

In [158]:
# entity fixed effects
model3 = PanelOLS.from_formula(
    "lwage ~ exper + EntityEffects",
    data=data,
)
result3 = model3.fit(cov_type="clustered", cluster_entity=True)

# time fixed effects
model4 = PanelOLS.from_formula(
    "lwage ~ exper + married + black + TimeEffects",
    data=data,
)
result4 = model4.fit(cov_type="clustered", cluster_time=True)

# time and entity fixed effects
model5 = PanelOLS.from_formula(
    "lwage ~ exper + EntityEffects + TimeEffects",
    data=data,
)
result5 = model5.fit(cov_type="clustered", cluster_entity=True, cluster_time=True)

### Fama-MacBeth

Run cross-sectional regressions and then use $t$ tests for the means of the time series of cross-sectional coefficients.

In [159]:
# A small data set with acc=accruals and agr=asset growth, 
# monthly data since 2010, roughly 2,000 stocks per month.

data = pd.read_csv("https://www.dropbox.com/s/012c6y4gxsxss6y/ghz.csv?dl=1", parse_dates=["date"])
data = data.sort_values(by=['permno', 'date'])
data = data.set_index(["permno", "date"])

In [160]:
# winsorize and standardize cross sections

data.agr = np.log(1+data.agr)

def winsorize(ser):
    return ser.clip(lower=ser.quantile(0.01), upper=ser.quantile(0.99))

for char in ["acc", "agr"]:
    data[char] = data.groupby("date")[char].apply(winsorize)
    data[char] = data.groupby("date")[char].apply(lambda x: x / x.std())



C:\Users\kerry\AppData\Local\Temp\ipykernel_20000\462842168.py:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data[char] = data.groupby("date")[char].apply(winsorize)
C:\Users\kerry\AppData\Local\Temp\ipykernel_20000\462842168.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data[char] = d

In [161]:
# run Fama-MacBeth

model6 = FamaMacBeth.from_formula("ret ~ acc + agr + 1", data=data)
result6 = model6.fit(cov_type="kernel", kernel="Bartlett", bandwidth=12)

In [162]:
# Fama-MacBeth "by hand" 

# cross-sectional regression function
def xreg(df):
    model = smf.ols("ret ~ acc + agr", data=df)
    result = model.fit()
    return result.params

# apply to each cross-section to get time series of coefficients
fm = data.groupby('date').apply(xreg)

# run t-tests by OLS to get Newey-West standard errors
model7a = smf.ols("acc ~ 1", data=fm)
result7a = model7a.fit(cov_type='HAC', cov_kwds={"kernel": "bartlett", "maxlags": 12})

model7b = smf.ols("agr ~ 1", data=fm)
result7b = model7b.fit(cov_type='HAC', cov_kwds={"kernel": "bartlett", "maxlags": 12})

## Rolling Window Betas

For Fama-MacBeth, we run cross-sectional regressions at each date.  For this exercise, we will run time-series regressions for each entity (stock).  We'll run the time-series regressions over rolling windows. 

The time series regressions are for stock returns on Fama-French factors.  It is common to use 60 months as the window but to include all stock/months for which 24 past months were available in the prior 60 months.  We do that with window=60, min_nobs=24, and expanding=True.

The RollingOLS function crashes if you specify a window size that is larger than the number of rows in the data frame.  So, we construct a function to "pass" if the number of rows is less than 24 and specify the window size as the smaller of 60 and the number of rows.

In [163]:
ff = pdr.DataReader("F-F_Research_Data_Factors", "famafrench", start=2000)[0] / 100
data = data.merge(ff, left_on='date', right_index=True) 
data = data.rename(columns={"Mkt-RF": "Mkt_RF"})
data["ret_RF"] = data.ret - data.RF
data = data.dropna(subset=["ret_RF", "Mkt_RF", "SMB", "HML"])

In [164]:
def rolling_betas(df):
    n = df.shape[0]
    if n >= 24:
        data = df.set_index("date") 
        model = RollingOLS.from_formula(
            "ret_RF ~ Mkt_RF + SMB + HML",
            window=min(n, 60),
            min_nobs=24,
            expanding=True,
            data=data
        )
        result = model.fit()
        return result.params[['Mkt_RF', 'SMB', 'HML']].dropna()
    else:
        pass
    
betas = data.reset_index().groupby("permno").apply(rolling_betas)

C:\Users\kerry\AppData\Local\Temp\ipykernel_20000\998454947.py:17: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  betas = data.reset_index().groupby("permno").apply(rolling_betas)


## Vector Auto-Regressions

In [165]:
model = VAR(ff)
result = model.fit()

## Saving Results

In [166]:
data = pd.read_csv("WAGE1_revised.csv")

# transformations of variables
model1 = smf.ols("wage ~ female", data=data)
result1 = model1.fit()

# multivariate
model2 = smf.ols("wage ~ female + educ + female*educ", data=data)
result2 = model2.fit()

# saving to Excel
pd.DataFrame(result2.summary().tables[1]).to_excel("excelfile.xlsx")


PermissionError: [Errno 13] Permission denied: 'excelfile.xlsx'

In [ ]:
# saving to tex

pystout(
    models=[result1, result2], 
    file="texfile.tex",
    endog_names = ["wage", "wage"],
    exogvars=[
        'female', 
        'educ', 
        'female:educ', 
        ],
    stars={0.1: "*", 0.05: "**", 0.01: "***"},
    addnotes=["$^*p<0.1$, $^{**}p<0.05$, $^{***}p<0.01$"],
    modstat={"nobs": "Obs", "rsquared_adj": "Adj $R^2$"},
    title="Wage Equation",
    label="tab:wage"

)

c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\pystout\pystout.py:377: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(pd.DataFrame([r],index=[value]))
c:\Users\kerry\AppData\Local\Programs\Python\Python310\lib\site-packages\pystout\pystout.py:377: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  options = options.append(pd.DataFrame([r],index=[value]))
